In [11]:
import pandas as pd

In [34]:
exports_zip = '../data/exp.zip'
imports_zip = '../data/imp.zip'
!zcat $exports_zip | head -n 4
!zcat $imports_zip | head -n 4

"CO_ANO";"CO_MES";"CO_NCM";"CO_UNID";"CO_PAIS";"SG_UF_NCM";"CO_VIA";"CO_URF";"QT_ESTAT";"KG_LIQUIDO";"VL_FOB"
"1997";"02";"84659110";"11";"548";"SP";"01";"0817800";2;1295;4294
"1997";"01";"84798999";"11";"158";"SP";"04";"0817700";6;352;1144
"1997";"01";"84819090";"10";"764";"SP";"04";"0817600";0;0;281

gzip: stdout: Broken pipe
"CO_ANO";"CO_MES";"CO_NCM";"CO_UNID";"CO_PAIS";"SG_UF_NCM";"CO_VIA";"CO_URF";"QT_ESTAT";"KG_LIQUIDO";"VL_FOB"
"1997";"12";"90072090";"11";"399";"AM";"01";"0227600";15;105;7184
"1997";"12";"30062000";"10";"767";"MG";"04";"0617600";6104;4855;563241
"1997";"12";"85331000";"11";"399";"SP";"04";"0817700";1000119;61;11826

gzip: stdout: Broken pipe


In [35]:
def quick_inspect(file_name, size):
    for chunk in pd.read_csv(file_name, delimiter=";", chunksize=size):
        display(chunk.head())
        display(chunk.info())
        break

size = 1000
for zip_file in [exports_zip, imports_zip]:
    quick_inspect(zip_file, size)

,CO_ANO,CO_MES,CO_NCM,CO_UNID,CO_PAIS,SG_UF_NCM,CO_VIA,CO_URF,QT_ESTAT,KG_LIQUIDO,VL_FOB
0,1997,2,84659110,11,548,SP,1,817800,2,1295,4294
1,1997,1,84798999,11,158,SP,4,817700,6,352,1144
2,1997,1,84819090,10,764,SP,4,817600,0,0,281
3,1997,2,83111000,10,97,SP,7,145200,0,400,5025
4,1997,2,84829120,10,63,RS,4,1017600,3000,3,360


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 11 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   CO_ANO      1000 non-null   int64 
 1   CO_MES      1000 non-null   int64 
 2   CO_NCM      1000 non-null   int64 
 3   CO_UNID     1000 non-null   int64 
 4   CO_PAIS     1000 non-null   int64 
 5   SG_UF_NCM   1000 non-null   object
 6   CO_VIA      1000 non-null   int64 
 7   CO_URF      1000 non-null   int64 
 8   QT_ESTAT    1000 non-null   int64 
 9   KG_LIQUIDO  1000 non-null   int64 
 10  VL_FOB      1000 non-null   int64 
dtypes: int64(10), object(1)
memory usage: 86.1+ KB


None

,CO_ANO,CO_MES,CO_NCM,CO_UNID,CO_PAIS,SG_UF_NCM,CO_VIA,CO_URF,QT_ESTAT,KG_LIQUIDO,VL_FOB
0,1997,12,90072090,11,399,AM,1,227600,15,105,7184
1,1997,12,30062000,10,767,MG,4,617600,6104,4855,563241
2,1997,12,85331000,11,399,SP,4,817700,1000119,61,11826
3,1997,12,84689090,10,249,SP,4,817600,20,20,414
4,1997,12,34049029,10,249,SP,1,817800,2915,1976,11471


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 11 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   CO_ANO      1000 non-null   int64 
 1   CO_MES      1000 non-null   int64 
 2   CO_NCM      1000 non-null   int64 
 3   CO_UNID     1000 non-null   int64 
 4   CO_PAIS     1000 non-null   int64 
 5   SG_UF_NCM   1000 non-null   object
 6   CO_VIA      1000 non-null   int64 
 7   CO_URF      1000 non-null   int64 
 8   QT_ESTAT    1000 non-null   int64 
 9   KG_LIQUIDO  1000 non-null   int64 
 10  VL_FOB      1000 non-null   int64 
dtypes: int64(10), object(1)
memory usage: 86.1+ KB


None